# 1) Definir caminhos (ABFSS)

In [0]:
BASE = "abfss://imoveis@datalakealura.dfs.core.windows.net"
BRONZE_DIR  = f"{BASE}/bronze"
BRONZE_PATH = f"{BRONZE_DIR}/dataset_imoveis"

SILVER_DIR  = f"{BASE}/silver"
SILVER_PATH = f"{SILVER_DIR}/dataset_imoveis"


# 2) Conferir acesso à pasta bronze

In [0]:
dbutils.fs.ls(BRONZE_DIR)


# 3) Ler o Delta da Bronze

In [0]:
df = spark.read.format("delta").load(BRONZE_PATH)
display(df)


# 4) Abrir os campos do JSON em colunas

In [0]:
display(df.select("anuncio.*", "id"))


In [0]:
dados_detalhados = df.select("anuncio.*", "anuncio.endereco.*", "id")
display(dados_detalhados)


# 5) Remover colunas (caracteristicas e endereco)

In [0]:
from pyspark.sql.functions import col

dados_detalhados = df.select("anuncio.*", "anuncio.endereco.*")  # sem "id" aqui



In [0]:
df_silver_raw = dados_detalhados.drop("caracteristicas", "endereco")
display(df_silver_raw)



### Ajustar arrays para valores simples (primeiro elemento)

In [0]:
from pyspark.sql.functions import element_at

df_silver = (
    df_silver_raw
    .withColumn("banheiros", element_at(col("banheiros"), 1))
    .withColumn("quartos", element_at(col("quartos"), 1))
    .withColumn("suites", element_at(col("suites"), 1))
    .withColumn("vaga", element_at(col("vaga"), 1))
    .withColumn("area_util", element_at(col("area_util"), 1))
    .withColumn("area_total", element_at(col("area_total"), 1))
)
display(df_silver)


# 6) Salvar na Silver em Delta (overwrite)

In [0]:

(df_silver.write
 .format("delta")
 .mode("overwrite")
 .save(SILVER_PATH))


# 7) Validar que salvou

### 7.1 Listar o diretório

In [0]:
dbutils.fs.ls(SILVER_PATH)


### 7.2 Ler de volta como Delta

In [0]:
silver_lido = spark.read.format("delta").load(SILVER_PATH)
display(silver_lido)
